In [2]:
import numpy as np
import pandas as pd
import math
from numpy.linalg import inv
#hello this is khai 

In [4]:
# Read the Excel file
#basecase = pd.ExcelFile('C:\\Users\\jingy\\Documents\\EE454Lab\\system_basecase.xlsx')
# khai's file location
basecase = pd.ExcelFile("C:\\Users\\kayla\\OneDrive - UW\\EE 454\\myrepo\\EE454Project\\system_basecase - Copy.xlsx")

# Read the busdata sheet
busdata = pd.read_excel(basecase, 'BusData')
# Read the linedata sheet
linedata = pd.read_excel(basecase, 'LineData')

# Convert to numpy arrays for later use
busdata_np = np.array(busdata)
linedata_np = np.array(linedata)

# # print data to test if it reads correctly
#print("Bus Data:", busdata_np.shape)
pd.DataFrame(busdata_np)




,0,1,2,3,4,5
0,1,0,0,S,0,1
1,2,40,30,D,0,1


In [5]:
num_buses = len(busdata_np)

In [6]:
print("Line Data:", linedata_np.shape)
pd.DataFrame(linedata_np)

Line Data: (1, 6)


,0,1,2,3,4,5
0,1.0,2.0,0.0,0.2,0.0,95.0


In [7]:
def create_Y_matrix(busdata, linedata):

    # Initialize the admittance matrix Y with all zeros
    Y = np.zeros((num_buses, num_buses), dtype=complex)

    # Process each line
    for line in linedata:
        # Get the bus number also take account the 0 index based numbering
        from_bus = int(line[0]) - 1
        to_bus = int(line[1]) - 1
        R = line[2]
        X = line[3]
        # Get the half-line shunt susceptance
        B_shunt = line[4] / 2

        Z = complex(R, X)
        # Get the line admittance
        Y_line = 1 / Z

        # Add each admittance component to the matrix
        Y[from_bus, from_bus] += Y_line + complex(0, B_shunt)
        Y[to_bus, to_bus] += Y_line + complex(0, B_shunt)
        Y[from_bus, to_bus] -= Y_line
        Y[to_bus, from_bus] -= Y_line

    # (commented out as optional) 
    # Get the conductance (G) and susceptance (B) seperately
#     G = np.real(Y)
#     B = np.imag(Y)

    return Y

In [10]:
# Get the admittence matrix
Y = create_Y_matrix(busdata_np, linedata_np)
G = np.real(Y)
B = np.imag(Y)
print("Admittance Matrix:\n")

pd.DataFrame(Y)


Admittance Matrix:



,0,1
0,0.0-5.0j,0.0+5.0j
1,0.0+5.0j,0.0-5.0j


In [11]:
V_mag = busdata_np[:, 5]
V_ang = np.zeros(num_buses) #radians
busnum = busdata_np[:,0]
P_Load = busdata_np[:, 1]
Q_Load = busdata_np[:, 2]
P_Gen = busdata_np[:, 4]
bus_type = busdata_np[:, 3]

slack = None
PQind = []
PVind = []

for i in range(num_buses):


    # initialize V based on bus type
    if bus_type[i] == 'S':
        slack = i
    elif bus_type[i] == 'G':
        PVind.append(i)
    elif bus_type[i] == 'D':
        PQind.append(i)

In [13]:
print("Vmag:" + str(V_mag))
print("Vang:" + str(V_ang))

Vmag:[1 1]
Vang:[0. 0.]


In [14]:
#formulas in textbook are written as 1 being Slack, 2 to m being PV (gen), and m+1 to N being PQ 

def Pmismatches (vpu, G, B, theta, pgen, pload):
    
    #initialize array
    Pmis = np.zeros(num_buses)
    
    #for each bus...
    for k in range(num_buses):
        
        #initialize a sum for all buses
        sum = 0
        for i in range(num_buses):
            sum +=  vpu[i]*vpu[k]*(G[k, i]*math.cos(theta[k]-theta[i])+B[k, i]*
                                   math.sin((theta[k]-theta[i]))) 
            
        #add sum to load-gen (divided by 100 M for per-unit value) for each bus
        Pmis[k] = sum + pload[k]/100. - pgen[k]/100.
        
    return Pmis


def Qmismatches (vpu, G, B, theta, qload):
    #initialize array

    Qmis = np.zeros(num_buses)
    
    #for each bus...
    for k in range(num_buses):
        sum = 0
        
        #initialize a sum for all buses

        for i in range(num_buses):
            sum +=  vpu[i]*vpu[k]*(G[k, i]*math.sin((theta[k]-theta[i]))-B[k, i]*
                                   math.cos((theta[k]-theta[i])))
        
        #add sum to load (divided by 100 M for per-unit value) for each bus

        Qmis[k] = sum  + qload[k]/100.
        
    return Qmis


    

In [16]:
print(Pmismatches(V_mag, G, B, V_ang, P_Gen, P_Load))
print(Qmismatches(V_mag, G, B, V_ang, Q_Load))

[0.  0.4]
[0.  0.3]


In [17]:
#mostly done ? 
def HMatrix (G, B, vpu, theta):
    #HMatrix will iterate P(row) from buses 2 to N and theta(col) from buses 2 to N
    
    #rows and cols are the index numbers of buses to be iterated
    rows = np.arange(1, num_buses, dtype=int)
    cols = np.arange(1, num_buses, dtype=int)
  
    
    #empty H array
    H = np.zeros((len(rows), len(cols)))
    
    #r represents the 
    r = 0 
    
    for k in rows:
        sum = 0
        c = 0
        matchBus = False
        matchCol = None
        
        for i in range(0, num_buses):
            

            
            
            
            if (i != k and i in cols):
                H[r,c] = vpu[k]*vpu[i]*(G[k, i]*math.sin((theta[k]-theta[i]))-B[k, i]*
                                            math.cos((theta[k]-theta[i])))
#                 print("H has been printed at " + str(r) + " " + str(c))
            elif (i == k and i in cols):
                    matchBus = True
                    matchCol = c
                
                
            if (i != k):
                sum += vpu[k]*vpu[i]*(-1*G[k, i]*math.sin((theta[k]-theta[i]))+B[k, i]*
                                            math.cos((theta[k]-theta[i])))
            
            if i in cols:
                c += 1
                
            
                
        if (matchBus):     
        
            H[r,matchCol] = sum
        
        r += 1
        
    return H

In [18]:
def MMatrix (G, B, vpu, theta):
    #MMatrix will iterate P(row) from buses 2 to N and V(col) for PQ buses
    
    #rows and cols are the index numbers of buses to be iterated
    rows = np.arange(1, num_buses, dtype=int)
    cols = PQind
    
    
    #optional check for index values
#     print("rows" + str(rows))
#     print("cols" + str(cols))
    
    #empty M array
    M = np.zeros((len(rows), len(cols)))
    
    
    r = 0 
    
    for k in rows:
        sum = 0
        c = 0
        
        matchBus = False
        matchCol = None
        
        #print("k is " + str(k))
        #print("hrow is " + str(hr))
        for i in range(0, num_buses):
            
            
            #print("i is " + str(i) + "and mcol is " + str(c))

            
            
            
            if (i != k and i in cols):
                M[r,c] = vpu[k]*(G[k, i]*math.cos((theta[k]-theta[i]))+B[k, i]*
                                            math.sin((theta[k]-theta[i])))
#                 print("M has been printed at " + str(r) + " " + str(c))
            elif (i == k and i in cols):
                    matchBus = True
                    matchCol = c
                    
                
            if (i != k):
                sum += vpu[i]*(G[k, i]*math.cos((theta[k]-theta[i]))+B[k, i]*
                                            math.sin((theta[k]-theta[i])))
            
            if i in cols:
                c += 1
                
        if (matchBus):     
        
            M[r,matchCol] = 2*G[k, k]*vpu[k] + sum
        
        r += 1
        
    return M

In [19]:
def NMatrix (G, B, vpu, theta):
    #NMatrix will iterate Q(row) for PQ buses and theta(col) for PQ and PV buses
    
    #rows and cols are the index numbers of buses to be iterated
    rows = PQind
    cols = np.arange(1, num_buses, dtype=int)
    
    #optional check for index values
#     print("rows" + str(rows))
#     print("cols" + str(cols))
    
    #empty N array
    N = np.zeros((len(rows), len(cols)))
    
    
    r = 0 
    
    for k in rows:
        sum = 0
        c = 0
        
        matchBus = False
        matchCol = None
#         print("k is " + str(k))
        #print("hrow is " + str(hr))
        for i in range(0, num_buses):
            
#             print("i is " + str(i) + "and Ncol is " + str(c))

            
            
            
            if (i != k and i in cols):
                N[r,c] = vpu[k]*vpu[i]*(-1*G[k, i]*math.cos((theta[k]-theta[i]))-B[k, i]*
                                            math.sin((theta[k]-theta[i])))
#                 print("N has been printed at " + str(r) + " " + str(c))
            elif (i == k and i in cols):
                    matchBus = True
                    matchCol = c     
            
                
            if (i != k):
                sum += vpu[k]*vpu[i]*(G[k, i]*math.cos((theta[k]-theta[i]))+B[k, i]*
                                            math.sin((theta[k]-theta[i])))
            
            if i in cols:
                c += 1
                
        if (matchBus):     
        
            N[r,matchCol] = sum
        
        r += 1
        
    return N

In [20]:
def LMatrix (G, B, vpu, theta):
    #LMatrix will iterate Q(row) for PQ buses and V(col) for PQ buses
    
    #rows and cols are the index numbers of buses to be iterated
    rows = PQind
    cols = PQind
    
    #optional check for index values
#     print("rows" + str(rows))
#     print("cols" + str(cols))
    
    #empty L array
    L = np.zeros((len(rows), len(cols)))
    
    
    r = 0 
    
    for k in rows:
        sum = 0
        c = 0
        
        matchBus = False
        matchCol = None
        
        #print("k is " + str(k))
        #print("hrow is " + str(hr))
        for i in range(0, num_buses):
            
            #print("i is " + str(i) + "and lcol is " + str(c))

            
            
            
            if (i != k and i in cols):
                L[r,c] = vpu[k]*(G[k, i]*math.sin((theta[k]-theta[i]))-B[k, i]*
                                            math.cos((theta[k]-theta[i])))
#                 print("L has been printed at " + str(r) + " " + str(c))

            elif (i == k and i in cols):
                    matchBus = True
                    matchCol = c
                    
            if (i != k):
                sum += vpu[i]*(G[k, i]*math.sin((theta[k]-theta[i]))-B[k, i]*
                                            math.cos((theta[k]-theta[i])))
            
            if i in cols:
                c += 1
                
        if (matchBus):     
        
            L[r,matchCol] =  -2*B[k, k]*vpu[k] + sum
        
        r += 1
        
    return L

In [21]:
def build_jacobian(G, B, vpu, theta):
    mat1 = np.concatenate((HMatrix(G, B, vpu, theta),NMatrix(G, B, vpu, theta)), axis=0)
    mat2 = np.concatenate((MMatrix(G, B, vpu, theta),LMatrix(G, B, vpu, theta)), axis=0)

    J = np.concatenate((mat1, mat2), axis=1)
    return J

In [22]:
J = build_jacobian(G, B, V_mag, V_ang)
pd.DataFrame(J)

,0,1
0,5.0,0.0
1,0.0,5.0


In [23]:
print(PQind)

[1]


In [26]:
def NR (vpu, G, B, theta, pgen, pload, qload):
    
    twoToN =  np.arange(1, num_buses, dtype=int)
    
    allPmis = Pmismatches(vpu, G, B, theta, pgen, pload)
    
    allQmis =  Qmismatches(vpu, G, B, theta, qload)
    
    newTheta = np.copy(theta)
    newV = np.copy(vpu)
    
    
#     firstPMismatch = allPmis[twoToN]
#     firstQMismatch = allQmis[PQind]
    
    
    
    currentPMismatch = allPmis
    currentQMismatch = allQmis
    
    print("P mismatch is " + str(currentPMismatch))
    print("Q mismatch is " + str(currentQMismatch))

    notSmall = True

    #while(notSmall):
    for i in range(2):
        J = build_jacobian(G, B, newV, newTheta)
        delta = np.matmul(inv(J), np.concatenate((currentPMismatch[twoToN], currentQMismatch[PQind])))

        print("delta is " + str(delta))
#         print(delta[0:len(twoToN)])
#         print(delta[len(twoToN):])


        print("newV is " + str(newV) )

        print("newT is " + str(newTheta))
        
        newTheta[twoToN] = newTheta[twoToN]-(delta[0:len(twoToN)])

        newV[PQind] = newV[PQind] - delta[len(twoToN):]

        
        currentPMismatch = Pmismatches(newV, G, B, newTheta, pgen, pload)
        currentQMismatch = Qmismatches(newV, G, B, newTheta, qload)

        print("P mismatch is " + str(currentPMismatch))
        print("Q mismatch is " + str(currentQMismatch))
        print("newV is " + str(newV) )

        print("newT is " + str(newTheta) )

        if(np.all(np.abs(currentPMismatch) <= .1) and np.all(np.abs(currentQMismatch) <= .1)):
            notSmall = False
            
        print(notSmall)

    
    return newV, newTheta
        

In [27]:
newV, newTheta = NR(V_mag, G, B, V_ang, P_Gen, P_Load, Q_Load)

P mismatch is [0.  0.4]
Q mismatch is [0.  0.3]
delta is [0.08 0.06]
newV is [1 1]
newT is [0. 0.]
P mismatch is [0.37559906 0.02440094]
Q mismatch is [0.31503198 0.03303198]
newV is [1 0.94]
newT is [ 0.   -0.08]
True
delta is [0.00588903 0.00798097]
newV is [1 0.94]
newT is [ 0.   -0.08]
P mismatch is [3.99759135e-01 2.40865114e-04]
Q mismatch is [0.35708287 0.00038024]
newV is [1 0.9320190301161808]
newT is [ 0.         -0.08588903]
True


In [213]:
allPmis = Pmismatches(newV, G, B, newTheta, P_Gen, P_Load)
allPmisold = Pmismatches(V_mag, G, B, V_ang, P_Gen, P_Load)
    

pd.DataFrame(allPmis)

,0
0,1.063600e+00
1,3.885781e-16
2,4.440892e-16
3,-3.330669e-16
4,7.202572e-15
5,-6.661338e-16
6,4.996004e-16
7,-3.053113e-16
8,2.498002e-16
9,3.885781e-16


In [181]:
a = np.arange(3)
print(a)
print(np.any(a>8))

ab = ([True, True, False])
a[ab] = [10,10]
print(a.shape)

[0 1 2]
False
(3,)


In [214]:
pd.DataFrame(allPmisold)

,0
0,0.257232
1,-0.027341
2,0.096987
3,0.572974
4,0.028627
5,0.065380
6,0.292280
7,0.052382
8,-0.108747
9,-0.136094
